In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

In [ ]:
def VisualizeBinaryClassificationData(features, labels):
    # function to visualize binary classification data. 
    # function only visualizes the first two features in 2D cartesian grid. 
    # Samples with label = 1 are depicted with "+" and those with label = 0 with "-"
    plt.figure()
    pos_rows = labels > 0
    neg_rows = labels <= 0
    plt.plot(features[pos_rows,0],features[pos_rows,1],'+',markersize=10,mew=2)
    plt.plot(features[neg_rows,0],features[neg_rows,1],'_',markersize=10,mew=2)
    plt.grid('on')
    plt.xlabel('feature1',fontsize=16), plt.ylabel('feature2',fontsize=16)
    plt.show()

In [ ]:
def VisBinClassDecisionBoundaries(model, xlims=[-30,30], ylims=[-30,30], h=0.05, features=None, 
                                  labels=None, alpha=0.25, title=None):
    # function visualizes decision boundaries using color plots
    # model is the classification model that can be any model in the scikit-learn package
    
    # creating meshgrid for different values of features
    xx, yy = np.meshgrid(np.arange(xlims[0], xlims[1], h), np.arange(ylims[0], ylims[1], h))
    # extracting predictions at different points in the mesh
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # plotting the mesh
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired, alpha=alpha)
    plt.grid()
    
    # if the samples are given plot them on the same plot
    if (features is not None) and (labels is not None): 
        pos_rows = labels > 0
        neg_rows = labels <= 0
        plt.plot(features[pos_rows,0],features[pos_rows,1],'k+',markersize=10,mew=2)
        plt.plot(features[neg_rows,0],features[neg_rows,1],'r_',markersize=10,mew=2)
        plt.grid('on')
        
    plt.xlabel('feature1',fontsize=16), plt.ylabel('feature2',fontsize=16)
    if title is not None: 
        plt.title(title, fontsize=16)
    plt.show() 

<h2>Generalization accuracy: Training and test set separation</h2>

One of the most important concepts in machine learning is <b>generalization accuracy</b>. Generalization accuracy is the quantity that tells us how well the trained model will be able to predict labels for <b>unseen</b> data samples. If we consider an engineering application, it tells us how well our trained model will perform once it is launched and customers start using it. It is easy to imagine that estimating generalization accuracy is quite crucial. 

This brings us to another important topic: <b> separatation of training and test sets</b>. As we have seen in the previous sections, "learning" is the estimation of the model parameters using a set of data samples, i.e. training set. In the supervised learning framework -- all the problems we have seen so far are supervised -- training set includes samples with both features and labels. Learning is formulated as estimating the set of parameters that yields the highest accuracy in predicting the labels using the features on the samples in the training set. It is easy to see that for another set of samples these parameters are not necessarily the optimal ones. Therefore, the prediction accuracy in the training set can be higher than the prediction accuracy in another set of samples. As a result, to estimate generalization accuracy one need to use a different set of samples than the training set. 

The most straightforward way to estimate generalization accuracy is to have two sets of parameters - one training and one testing. Training set is used to learn the model and test set is used to estimate the generalization accuracy. It is critical to never mix training and test sets. Let us see for an example how these sets are used. We use the same binary classification example we have used before. 

In [ ]:
# Reading data
training_features = np.loadtxt('data/train_genacc_features.txt')
training_labels = np.loadtxt('data/train_genacc_labels.txt')

# Plotting
VisualizeBinaryClassificationData(training_features, training_labels)

Let us train two different models on this training set and estimate prediction accuracy on the training sets. For this, we can use the basic logistic regression model and a more complicated model, 2 layer neural network. 

(For the sake of illustration, I am specifically using small number of training samples.)

In [ ]:
# We import the necessary modules to perform logistic regression
from sklearn import linear_model
# We create an object that can do logistic regression
logr = linear_model.LogisticRegression()
# We use the data to estimate its parameters with the fit function
logr.fit(training_features, training_labels)

# Import the necessary models for MLP classifier
from sklearn.neural_network import MLPClassifier
# Create an object that will do the classification
nn2 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3,3), random_state=1)
# We use the training data to estimate the parameters. 
nn2.fit(training_features, training_labels)


Using the trained models we now compute the classification accuracy in the training set. 

In [ ]:
train_predict_logr = logr.predict(training_features)
train_predict_nn2  = nn2.predict(training_features)
clas_accuracy_logr = np.sum(train_predict_logr == training_labels) / training_features.shape[0]
clas_accuracy_nn2 = np.sum(train_predict_nn2 == training_labels) / training_features.shape[0]
print('Classification accuracy for logistic regression: {}'.format(clas_accuracy_logr))
print('Classification accuracy for neural networks with 2 HL: {}'.format(clas_accuracy_nn2))

Classification accuracies on the training set are quite high. 

Let us see how the boundaries look like. 

In [ ]:
VisBinClassDecisionBoundaries(logr, features=training_features, labels=training_labels, title='Logistic Regression')
VisBinClassDecisionBoundaries(nn2, features=training_features, labels=training_labels, title='Neural Network - 2 HL')

Indeed the class boundary the neural network model constructs looks complicated. Let us see quantify the classification accuracy on a separate test set. 

In [ ]:
test_features = np.loadtxt('data/test_genacc_features.txt')
test_labels = np.loadtxt('data/test_genacc_labels.txt')
test_predict_logr = logr.predict(test_features)
test_predict_nn2  = nn2.predict(test_features)
clas_accuracy_logr = np.sum(test_predict_logr == test_labels) / test_features.shape[0]
clas_accuracy_nn2 = np.sum(test_predict_nn2 == test_labels) / test_features.shape[0]
print('Number of test samples: {}'.format(test_features.shape[0]))
print('Classification accuracy for logistic regression: {}'.format(clas_accuracy_logr))
print('Classification accuracy for neural networks with 2 HL: {}'.format(clas_accuracy_nn2))

The accuracy on this separate set is much lower than the one on the training set. For logistic regression the difference is small but for the neural network model the difference is substantial. 

This behavior is not uncommon and is even more important for high dimensional data, which we will see shortly. 

High discrepancy between prediction accuracy on training and test sets indicate a phenomenon called <b>over-fitting</b>. When the number of model parameters is high relative to the number of samples in the training set, the parameter estimation during learning might focus on noise in addition to class differences to achieve the best possible prediction accuracy on the training samples. For the neural network example, we saw that the model achieved more than 90% accuracy on the training set. When we visualized the prediction boundary we saw that the class separation learned "too much". As a result, prediction accuracy on the test set was much lower than the one on the training set. 


<h3> Validation set </h3>

We have not seen validation set so far and for the sake of time we will not delve into it. However, it is important to know it as well. 

Most models have hyper-parameters that needs to be set by users. For instance, when designing a neural network model, users need to decide on the architecture, e.g. number of layers, number of nodes in each layer, ... For different models these parameters change. Validation set is often used to set these parameters in a principled way. 

One good approach for setting the hyper-parameters is to choose the values that yield highest generalization accuracy. We have already seen that using training set to estimate generalization accuracy is not a good idea. In order to tackle this, validation set is used. For each setting of hyper-parameters, models are trained using the training set and generalization accuracy is estimated using the validation set. Validation set is always kept apart from the training set. The hyper-parameter values that yield the highest prediction accuracy on the validation set is then used for the final model. 

In order to estimate the generalization accuracy of the final model, we still need a test set that is different than both the training and validation sets. It is critical that these three sets have no overlap. 

<h2> Cross validation </h2>

Having two or three (in case of validation set) separate datasets may not be feasible for some applications, where available data is limited. In these cases, dividing the already small sample sizes into training and test sets might yield very small training samples and create problems in the estimation of the model parameters. 

To address small sample sizes, an imperfect but widely used approach is cross-validation. In cross-validation the data is divided into different partitions, for instance $K$. The specific instance of this cross-validation is named K-fold cross-validation. The model is trained and prediction accuracy estimated $K$ times. Each time $K-1$ partitions of the dataset is used for training and the remaining partition is used to compute prediction accuracy. In the next round, another partition is set aside. At the end, each test sample is used once as a test sample. The final generalization accuracy is computed as the average of the individual runs. 

In scikit-learn this is already implemented. Let us see how it is used. 

In [ ]:
# import the required function to perform 5-fold stratified cross-validation
# in stratified K-fold cross validation in each fold the ratio of the 
# number of different classes is the same as the entire dataset. 
from sklearn.model_selection import StratifiedKFold
# creating an object to create partitions for the 5 fold cross validation
skf = StratifiedKFold(n_splits=5)

# in this for loop we go over different partitions. 
# "skf" object produces indices of samples for different folds
for trainind, testind in skf.split(training_features, training_labels):
    print("Training indices: {}".format(trainind))
    print("Test indices: {}".format(testind))


Let us now run the entire cross-validation experiment to estimate generalization accuracy and compare it with the training accuracy we computed above:

In [ ]:
# import the required function to perform 5-fold stratified cross-validation
# in stratified K-fold cross validation in each fold the ratio of the 
# number of different classes is the same as the entire dataset. 
from sklearn.model_selection import StratifiedKFold

# import the required function to compute classification accuracy
from sklearn.metrics import accuracy_score

# creating an object to create partitions for the 5 fold cross validation
numFolds = 5
skf = StratifiedKFold(n_splits=numFolds)

# creating a vector to hold accuracies of different folds: 
acc_vec_logr = np.zeros(numFolds)
acc_vec_nn2  = np.zeros(numFolds)
# in this for loop we go over different partitions. 
n = 0
for trainind, testind in skf.split(training_features, training_labels):
    # training both classification models using the training partitions of the dataset. 
    logr.fit(training_features[trainind,:], training_labels[trainind])
    nn2.fit(training_features[trainind,:], training_labels[trainind])
    
    # predictions in the test partition of each fold
    preds_cv_logr = logr.predict(training_features[testind,:])
    preds_cv_nn2 = nn2.predict(training_features[testind,:])
    
    # computing accuracy for the test partitions
    acc_vec_logr[n] = accuracy_score(training_labels[testind], preds_cv_logr)
    acc_vec_nn2[n]  = accuracy_score(training_labels[testind], preds_cv_nn2)
    n += 1

print("Accuracies at different folds:")
print("=============================")
print("Logistic Regression: {}".format(acc_vec_logr))
print("Neural Networks with 2 HL: {}".format(acc_vec_nn2))
print("\n")
print("Generalization accuracy estimates:")
print("=============================")
print("Logistic Regression: {}".format(np.mean(acc_vec_logr)))
print("Neural Networks with 2 HL: {}".format(np.mean(acc_vec_nn2)))

<b>Note:</b> Comparing generalization accuracy estimate computed with 5-fold cross-validation and training set accuracy shows the difference between these two approaches. 5-fold cross-validation is also not a perfect estimation technique however, this time it gets a much closer estimate to the accuracy on the separate test set than prediction accuracy on the training set. 

<b>Note:</b> Cross-validation is an experiment to estimate generalization accuracy. It is strategy to best make use of the available data. When creating the final model the learning algorithm is trained with all available training data and shipped. 

<h3>Alternatives</h3>
There are different alternatives to cross-validation. These include jack-knifing and boot-strapping. One of the most common approach is to run multiple cross-validation experiments each with random shuffling of the samples. 

We will not go into detail for all these alternatives. Websites of scikit-learn is a good starting point for these alternatives. 

<h3>Generalization accuracy when there are hyper-parameters</h3>
We mentioned validation set and estimation of hyper-parameters above. When there are hyper-parameters, estimation of the hyper-parameter is also considered part of learning. Therefore, any strategy to estimate generalization accuracy must take into account that part as well. One of the most common techniques for generalization accuracy estimation in the presence of hyper-parameters is <b>nested cross-validation</b>. We will not go into details of this topic but I advice you to read more about it if you plan to use machine learning models and optimise hyper-parameters.